In [1]:
## import libraries
import decimal
from google.cloud import bigquery
from google.api_core.exceptions import Conflict
import os
import pandas as pd

In [2]:
## load data
link_transactions = pd.read_csv('../data/link_transactions.csv')

In [3]:
## drop integer values and rename columns with floats
link_transactions.drop(columns=['TransactionFeeEth', 'SignedValue', 'RunningBalance'], inplace=True)
link_transactions['Value'] = link_transactions['Value'].astype('float64')/10**18
link_transactions.rename(inplace=True, columns={
    'TransactionFeeEthDecimal': 'TransactionFeeEth',
    'SignedValueDecimal': 'SignedValue',
    'RunningBalanceDecimal': 'RunningBalance'})

In [5]:
## set up bigquery table
# set credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../gcloud_service_creds.json'

# construct a BigQuery client object
client = bigquery.Client()

In [7]:
## create bq table

# define schema
schema = [
    bigquery.SchemaField("Name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("Address", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("From", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("To", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("ContractAddress", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("TokenName", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("TokenSymbol", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("Inbound", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("Outbound", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("Timestamp", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("Value", "BIGNUMERIC", mode="REQUIRED"),
    bigquery.SchemaField("SignedValue", "BIGNUMERIC", mode="REQUIRED"),
    bigquery.SchemaField("RunningBalance", "BIGNUMERIC", mode="REQUIRED"),
    bigquery.SchemaField("TransactionFeeEth", "BIGNUMERIC", mode="REQUIRED"),
]

# first create a project (banded-charmer-324215) in the console
project_name = 'banded-charmer-324215'

# create bigquery dataset
dataset_name = 'transactions'
try:
    client.create_dataset(dataset_name)
except Conflict as c:
    print("Dataset {}.{} already exists.".format(project_name, dataset_name))

# create table
table_id = "{}.{}.link_transactions".format(project_name, dataset_name)
table = bigquery.Table(table_id, schema=schema)
try:
    table = client.create_table(table)
    print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
except Conflict as c:
    print("Table {}.{}.{} already exists.".format(table.project, table.dataset_id, table.table_id))

Dataset banded-charmer-324215.transactions already exists.
Created table banded-charmer-324215.transactions.link_transactions


In [8]:
## push data to bigquery table

# set dataframe columns to match order of defined schema
link_transactions = link_transactions[[x.name for x in schema]]

# create bigquery load job
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_APPEND",
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True
)

bq_source_file = '../data/link_transactions_bq.csv'
link_transactions.to_csv(bq_source_file, index=False, header=True)

In [9]:

# load the csv into bigquery
with open(bq_source_file, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for table load to complete.

# looks like everything worked :)
print("Loaded {} rows into {}.{}.{}.".format(
    job.output_rows, table.project, table.dataset_id, table.table_id))

Loaded 3205 rows into banded-charmer-324215.transactions.link_transactions.


In [35]:
##### confirm bigquery rows
query = """
    SELECT * 
    FROM {}.{}.{}
    """.format(table.project, table.dataset_id, table.table_id)
query_results = client.query(query)

results_df = pd.DataFrame()
for row in query_results.result():
    sample = {}
    for key, value in row.items():
        sample[key] = value
    results_df = results_df.append(sample, ignore_index=True)

In [40]:
results_df.sort_values(['Name', 'Timestamp'])

,Name,Address,From,To,ContractAddress,TokenName,TokenSymbol,Inbound,Outbound,Timestamp,Value,SignedValue,RunningBalance,TransactionFeeEth
3169,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x4cadde3de133ccb22718ff3ac3b54b86760895dd,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,1.0,0.0,2021-06-23 01:50:19+00:00,100,100,100,0.001354314
3065,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x1b4c87add7a98d28910a0e1ab616d71aa4b4ce8b,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,1.0,0.0,2021-06-25 16:52:54+00:00,3000,3000,3100,0.0042227
0,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x89177b9c203ba0a9294aecf2f3806d98907bec6f,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-07-01 07:46:50+00:00,8.618913187938,-8.618913187938,3091.381086812062,0.000925984
1,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0xd9459cc85e78e0336adb349eabf257dbaf9d5a2b,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-07-01 15:40:24+00:00,5.33746003161,-5.33746003161,3086.043626780452,0.6810679
2,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x89177b9c203ba0a9294aecf2f3806d98907bec6f,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-07-01 15:40:24+00:00,0.032625,-0.032625,3086.0110017804523,0.6810679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3060,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x3fb4600736d306ee2a89edf0356d4272fb095768,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-08-09 01:19:38+00:00,12.35,-12.35,6747.21,0.002414745
3061,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x4fbefaf1bff0130945c61603b97d38dd6e21f5cf,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-08-16 11:41:59+00:00,20.15,-20.15,6727.06,0.001985457
3062,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x9efa0a617c0552f1558c95993aa8b8a68b3e709c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-08-16 15:01:51+00:00,154.54,-154.54,6572.52,0.005312439
3063,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x3fb4600736d306ee2a89edf0356d4272fb095768,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0.0,1.0,2021-08-22 22:42:39+00:00,12.35,-12.35,6560.17,0.001878135


In [41]:
link_transactions.sort_values(['Name', 'Timestamp'])

,Name,Address,From,To,ContractAddress,TokenName,TokenSymbol,Inbound,Outbound,Timestamp,Value,SignedValue,RunningBalance,TransactionFeeEth
0,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x4cadde3de133ccb22718ff3ac3b54b86760895dd,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,1,0,1624413019,100.000000,100.000000,100.000000,0.001354
1,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x1b4c87add7a98d28910a0e1ab616d71aa4b4ce8b,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,1,0,1624639974,3000.000000,3000.000000,3100.000000,0.004223
2,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x89177b9c203ba0a9294aecf2f3806d98907bec6f,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1625125610,8.618913,-8.618913,3091.381087,0.000926
3,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0xd9459cc85e78e0336adb349eabf257dbaf9d5a2b,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1625154024,5.337460,-5.337460,3086.043627,0.681068
4,aggregator_01,0x7813C776cD8EAb537028b4499d467b1f1B86b14C,0x7813c776cd8eab537028b4499d467b1f1b86b14c,0x89177b9c203ba0a9294aecf2f3806d98907bec6f,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1625154024,0.032625,-0.032625,3086.011002,0.681068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x3fb4600736d306ee2a89edf0356d4272fb095768,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1628471978,12.350000,-12.350000,6747.210000,0.002415
3201,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x4fbefaf1bff0130945c61603b97d38dd6e21f5cf,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1629114119,20.150000,-20.150000,6727.060000,0.001985
3202,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x9efa0a617c0552f1558c95993aa8b8a68b3e709c,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1629126111,154.540000,-154.540000,6572.520000,0.005312
3203,aggregator_17,0x989B836D68700DA948B5c04A65b3bBA39F400ad7,0x989b836d68700da948b5c04a65b3bba39f400ad7,0x3fb4600736d306ee2a89edf0356d4272fb095768,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,0,1,1629672159,12.350000,-12.350000,6560.170000,0.001878
